In [ ]:
import pandas as pd
import numpy as np
import math
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')

In [ ]:
df = pd.read_csv("/content/k-medoids_clustered_dataset.csv")
df.head()

,Year,Access to electricity (% of population),Agricultural land (% of land area),"Annual freshwater withdrawals, total (% of internal resources)",Arable land (% of land area),Forest area (% of land area),Electric power consumption (kWh per capita),Energy use (kg of oil equivalent per capita),Renewable electricity output (% of total electricity output),Renewable energy consumption (% of total final energy consumption),Population growth (annual %),GDP per capita (current US$),CO2 emissions (metric tons per capita),Cluster
0,2000,4.446891,57.945817,43.015907,11.779587,1.852782,1586.591120,985.730004,74.989094,44.99,1.443803,182.174037,0.055167,0
1,2001,9.294527,57.947350,43.015907,11.779587,1.852782,1587.375364,1011.679617,72.811460,45.60,0.742517,182.174037,0.055293,0
2,2002,14.133616,57.939684,43.015907,11.771921,1.852782,1649.718098,1034.410867,79.063971,37.83,6.449321,182.174037,0.066810,0
3,2003,18.971165,58.083805,43.015907,11.916042,1.852782,1738.666619,1010.524231,70.249729,36.66,7.541019,199.643228,0.073005,0
4,2004,23.814182,58.151266,43.015907,11.983503,1.852782,1841.168267,1121.869767,70.890841,44.24,3.933178,221.830531,0.054867,0


In [ ]:
df['Cluster'].value_counts()

Cluster
0    4391
1    1195
Name: count, dtype: int64

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X = df.drop(['Cluster'], axis=1)
y = df['Cluster']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print("X_train shape:", X_train.shape)
print("X_test shape:", X_test.shape)
print("y_train shape:", y_train.shape)
print("y_test shape:", y_test.shape)

X_train shape: (4468, 13)
X_test shape: (1118, 13)
y_train shape: (4468,)
y_test shape: (1118,)


#KNN

In [ ]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.model_selection import KFold

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
# Initialize KNN classifier
knn = KNeighborsClassifier()


# Fit the classifier on the training data
knn.fit(X_train, y_train)

# Predict on the test data
y_pred = knn.predict(X_test)

# Generate classification report
report = classification_report(y_test, y_pred, output_dict=True)

# Get unique labels from the 'Outcome' column
labels = y.unique()

# Dictionary to store evaluation metrics
scores = {label: {'specificity': None, 'sensitivity': None, 'precision': None, 'recall': None, 'f1_score': None, 'accuracy': None} for label in labels}

# Generate confusion matrix
cm = confusion_matrix(y_test, y_pred, labels=labels)

# Extract evaluation metrics for each label
for idx, label in enumerate(labels):
    tp = cm[idx, idx]  # True Positive
    fn = cm[idx, :].sum() - tp  # False Negative
    fp = cm[:, idx].sum() - tp  # False Positive
    tn = cm.sum() - (tp + fn + fp)  # True Negative

    specificity = tn / (tn + fp) if (tn + fp) > 0 else 0
    sensitivity = tp / (tp + fn) if (tp + fn) > 0 else 0
    precision = report[str(label)]['precision']
    recall = report[str(label)]['recall']
    f1_score = report[str(label)]['f1-score']
    accuracy = report['accuracy']

    scores[label]['specificity'] = specificity
    scores[label]['sensitivity'] = sensitivity
    scores[label]['precision'] = precision
    scores[label]['recall'] = recall
    scores[label]['f1_score'] = f1_score
    scores[label]['accuracy'] = accuracy

# Print evaluation metrics for each label
for label in labels:
    print(f"\nLabel '{label}' Scores:")
    print("Specificity:", scores[label]['specificity'])
    print("Sensitivity:", scores[label]['sensitivity'])
    print("Precision:", scores[label]['precision'])
    print("Recall:", scores[label]['recall'])
    print("F1 Score:", scores[label]['f1_score'])
    print("Accuracy:", scores[label]['accuracy'])

# Summarize results
print("\nOverall accuracy:", report['accuracy'])


Label '0' Scores:
Specificity: 0.9915966386554622
Sensitivity: 1.0
Precision: 0.9977324263038548
Recall: 1.0
F1 Score: 0.9988649262202043
Accuracy: 0.998211091234347

Label '1' Scores:
Specificity: 1.0
Sensitivity: 0.9915966386554622
Precision: 1.0
Recall: 0.9915966386554622
F1 Score: 0.9957805907172996
Accuracy: 0.998211091234347

Overall accuracy: 0.998211091234347


In [ ]:
# Initialize KNN classifier
knn = KNeighborsClassifier()

# Initialize KFold cross-validation
kf = KFold(n_splits=5, shuffle=True, random_state=42)

# Get unique labels from the 'Outcome' column
labels = y.unique()

# Dictionary to store evaluation metrics
avg_scores = {label: {'specificity': [], 'sensitivity': [], 'precision': [], 'recall': [], 'f1_score': [], 'accuracy': []} for label in labels}

# Perform KFold cross-validation
for i, (train_index, test_index) in enumerate(kf.split(X)):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    # Fit the classifier on the training data
    knn.fit(X_train, y_train)

    # Predict on the test data
    y_pred = knn.predict(X_test)

    # Generate classification report
    report = classification_report(y_test, y_pred, output_dict=True)

    # Generate confusion matrix
    cm = confusion_matrix(y_test, y_pred, labels=labels)

    # Extract evaluation metrics for each label
    for idx, label in enumerate(labels):
        tp = cm[idx, idx]  # True Positive
        fn = cm[idx, :].sum() - tp  # False Negative
        fp = cm[:, idx].sum() - tp  # False Positive
        tn = cm.sum() - (tp + fn + fp)  # True Negative

        specificity = tn / (tn + fp) if (tn + fp) > 0 else 0
        sensitivity = tp / (tp + fn) if (tp + fn) > 0 else 0
        precision = report[str(label)]['precision']
        recall = report[str(label)]['recall']
        f1_score = report[str(label)]['f1-score']
        accuracy = report['accuracy']

        avg_scores[label]['specificity'].append(specificity)
        avg_scores[label]['sensitivity'].append(sensitivity)
        avg_scores[label]['precision'].append(precision)
        avg_scores[label]['recall'].append(recall)
        avg_scores[label]['f1_score'].append(f1_score)
        avg_scores[label]['accuracy'].append(accuracy)

    # Print evaluation metrics for each label after each iteration
    print(f"\nFold {i+1}:")
    for label in labels:
        print(f"\nLabel '{label}' Scores:")
        print("Specificity:", avg_scores[label]['specificity'][-1])
        print("Sensitivity:", avg_scores[label]['sensitivity'][-1])
        print("Precision:", avg_scores[label]['precision'][-1])
        print("Recall:", avg_scores[label]['recall'][-1])
        print("F1 Score:", avg_scores[label]['f1_score'][-1])
        print("Accuracy:", avg_scores[label]['accuracy'][-1])

# Summarize results across all folds
print("\nAverage scores across all folds:")
for label in labels:
    print(f"\nLabel '{label}' Average Scores:")
    print("Specificity:", sum(avg_scores[label]['specificity']) / len(avg_scores[label]['specificity']))
    print("Sensitivity:", sum(avg_scores[label]['sensitivity']) / len(avg_scores[label]['sensitivity']))
    print("Precision:", sum(avg_scores[label]['precision']) / len(avg_scores[label]['precision']))
    print("Recall:", sum(avg_scores[label]['recall']) / len(avg_scores[label]['recall']))
    print("F1 Score:", sum(avg_scores[label]['f1_score']) / len(avg_scores[label]['f1_score']))
    print("Accuracy:", sum(avg_scores[label]['accuracy']) / len(avg_scores[label]['accuracy']))



Fold 1:

Label '0' Scores:
Specificity: 0.9915966386554622
Sensitivity: 1.0
Precision: 0.9977324263038548
Recall: 1.0
F1 Score: 0.9988649262202043
Accuracy: 0.998211091234347

Label '1' Scores:
Specificity: 1.0
Sensitivity: 0.9915966386554622
Precision: 1.0
Recall: 0.9915966386554622
F1 Score: 0.9957805907172996
Accuracy: 0.998211091234347

Fold 2:

Label '0' Scores:
Specificity: 1.0
Sensitivity: 0.9977142857142857
Precision: 1.0
Recall: 0.9977142857142857
F1 Score: 0.9988558352402745
Accuracy: 0.9982094897045658

Label '1' Scores:
Specificity: 0.9977142857142857
Sensitivity: 1.0
Precision: 0.9918032786885246
Recall: 1.0
F1 Score: 0.9958847736625513
Accuracy: 0.9982094897045658

Fold 3:

Label '0' Scores:
Specificity: 0.9957983193277311
Sensitivity: 0.9977246871444824
Precision: 0.9988610478359908
Recall: 0.9977246871444824
F1 Score: 0.9982925441092771
Accuracy: 0.9973142345568488

Label '1' Scores:
Specificity: 0.9977246871444824
Sensitivity: 0.9957983193277311
Precision: 0.991631799

#DT

In [ ]:
from sklearn.tree import DecisionTreeClassifier

# Initialize Decision Tree classifier
dt = DecisionTreeClassifier()

# Fit the classifier on the training data
dt.fit(X_train, y_train)

# Predict on the test data
y_pred = dt.predict(X_test)

# Generate classification report
report = classification_report(y_test, y_pred, output_dict=True)

# Get unique labels from the 'Outcome' column
labels = y.unique()

# Dictionary to store evaluation metrics
avg_scores = {label: {'specificity': None, 'sensitivity': None, 'precision': None, 'recall': None, 'f1_score': None, 'accuracy': None} for label in labels}

# Generate confusion matrix
cm = confusion_matrix(y_test, y_pred, labels=labels)

# Extract evaluation metrics for each label
for idx, label in enumerate(labels):
    print(f"\nLabel '{label}' Scores:")
    tp = cm[idx, idx]  # True Positive
    fn = cm[idx, :].sum() - tp  # False Negative
    fp = cm[:, idx].sum() - tp  # False Positive
    tn = cm.sum() - (tp + fn + fp)  # True Negative

    specificity = tn / (tn + fp) if (tn + fp) > 0 else 0
    sensitivity = tp / (tp + fn) if (tp + fn) > 0 else 0
    precision = report[str(label)]['precision']
    recall = report[str(label)]['recall']
    f1_score = report[str(label)]['f1-score']
    accuracy = report['accuracy']

    print("Specificity:", specificity)
    print("Sensitivity:", sensitivity)
    print("Precision:", precision)
    print("Recall:", recall)
    print("F1 Score:", f1_score)
    print("Accuracy:", accuracy)

# Summarize results
print("\nOverall accuracy:", report['accuracy'])


Label '0' Scores:
Specificity: 1.0
Sensitivity: 0.9955056179775281
Precision: 1.0
Recall: 0.9955056179775281
F1 Score: 0.9977477477477478
Accuracy: 0.9964189794091316

Label '1' Scores:
Specificity: 0.9955056179775281
Sensitivity: 1.0
Precision: 0.9826839826839827
Recall: 1.0
F1 Score: 0.9912663755458515
Accuracy: 0.9964189794091316

Overall accuracy: 0.9964189794091316


In [ ]:
from sklearn.tree import DecisionTreeClassifier

# Initialize Decision Tree classifier
dt = DecisionTreeClassifier()

# Initialize KFold cross-validation
kf = KFold(n_splits=5, shuffle=True, random_state=42)

# Get unique labels from the 'Outcome' column
labels = y.unique()

# Dictionary to store evaluation metrics
avg_scores = {label: {'specificity': [], 'sensitivity': [], 'precision': [], 'recall': [], 'f1_score': [], 'accuracy': []} for label in labels}

# Perform KFold cross-validation
for i, (train_index, test_index) in enumerate(kf.split(X)):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    # Fit the classifier on the training data
    dt.fit(X_train, y_train)

    # Predict on the test data
    y_pred = dt.predict(X_test)

    # Generate classification report
    report = classification_report(y_test, y_pred, output_dict=True)

    # Generate confusion matrix
    cm = confusion_matrix(y_test, y_pred, labels=labels)

    # Extract evaluation metrics for each label
    for idx, label in enumerate(labels):
        tp = cm[idx, idx]  # True Positive
        fn = cm[idx, :].sum() - tp  # False Negative
        fp = cm[:, idx].sum() - tp  # False Positive
        tn = cm.sum() - (tp + fn + fp)  # True Negative

        specificity = tn / (tn + fp) if (tn + fp) > 0 else 0
        sensitivity = tp / (tp + fn) if (tp + fn) > 0 else 0
        precision = report[str(label)]['precision']
        recall = report[str(label)]['recall']
        f1_score = report[str(label)]['f1-score']
        accuracy = report['accuracy']

        avg_scores[label]['specificity'].append(specificity)
        avg_scores[label]['sensitivity'].append(sensitivity)
        avg_scores[label]['precision'].append(precision)
        avg_scores[label]['recall'].append(recall)
        avg_scores[label]['f1_score'].append(f1_score)
        avg_scores[label]['accuracy'].append(accuracy)

    # Print evaluation metrics for each label after each iteration
    print(f"Fold {i+1}:")
    for label in labels:
        print(f"\nLabel '{label}' Scores:")
        print("Specificity:", avg_scores[label]['specificity'][-1])
        print("Sensitivity:", avg_scores[label]['sensitivity'][-1])
        print("Precision:", avg_scores[label]['precision'][-1])
        print("Recall:", avg_scores[label]['recall'][-1])
        print("F1 Score:", avg_scores[label]['f1_score'][-1])
        print("Accuracy:", avg_scores[label]['accuracy'][-1])

# Summarize results across all folds
print("\nAverage scores across all folds:")
for label in labels:
    print(f"\nLabel '{label}' Average Scores:")
    print("Specificity:", sum(avg_scores[label]['specificity']) / len(avg_scores[label]['specificity']))
    print("Sensitivity:", sum(avg_scores[label]['sensitivity']) / len(avg_scores[label]['sensitivity']))
    print("Precision:", sum(avg_scores[label]['precision']) / len(avg_scores[label]['precision']))
    print("Recall:", sum(avg_scores[label]['recall']) / len(avg_scores[label]['recall']))
    print("F1 Score:", sum(avg_scores[label]['f1_score']) / len(avg_scores[label]['f1_score']))
    print("Accuracy:", sum(avg_scores[label]['accuracy']) / len(avg_scores[label]['accuracy']))


Fold 1:

Label '0' Scores:
Specificity: 0.9873949579831933
Sensitivity: 1.0
Precision: 0.9966024915062288
Recall: 1.0
F1 Score: 0.998298355076574
Accuracy: 0.9973166368515206

Label '1' Scores:
Specificity: 1.0
Sensitivity: 0.9873949579831933
Precision: 1.0
Recall: 0.9873949579831933
F1 Score: 0.9936575052854123
Accuracy: 0.9973166368515206
Fold 2:

Label '0' Scores:
Specificity: 1.0
Sensitivity: 0.9988571428571429
Precision: 1.0
Recall: 0.9988571428571429
F1 Score: 0.9994282447112636
Accuracy: 0.999104744852283

Label '1' Scores:
Specificity: 0.9988571428571429
Sensitivity: 1.0
Precision: 0.9958847736625515
Recall: 1.0
F1 Score: 0.9979381443298969
Accuracy: 0.999104744852283
Fold 3:

Label '0' Scores:
Specificity: 1.0
Sensitivity: 1.0
Precision: 1.0
Recall: 1.0
F1 Score: 1.0
Accuracy: 1.0

Label '1' Scores:
Specificity: 1.0
Sensitivity: 1.0
Precision: 1.0
Recall: 1.0
F1 Score: 1.0
Accuracy: 1.0
Fold 4:

Label '0' Scores:
Specificity: 0.98
Sensitivity: 0.9988465974625144
Precision: 0.9

#Neural Network

In [ ]:
from sklearn.neural_network import MLPClassifier
# Initialize Neural Network classifier
nn = MLPClassifier(random_state=42)

# Fit the classifier on the training data
nn.fit(X_train, y_train)

# Predict on the test data
y_pred = nn.predict(X_test)

# Generate classification report
report = classification_report(y_test, y_pred, output_dict=True)

# Get unique labels from the 'Outcome' column
labels = y.unique()

# Dictionary to store evaluation metrics
scores = {label: {'specificity': 0, 'sensitivity': 0, 'precision': 0, 'recall': 0, 'f1_score': 0, 'accuracy': 0} for label in labels}

# Generate confusion matrix
cm = confusion_matrix(y_test, y_pred, labels=labels)

# Extract evaluation metrics for each label
for idx, label in enumerate(labels):
    tp = cm[idx, idx]  # True Positive
    fn = cm[idx, :].sum() - tp  # False Negative
    fp = cm[:, idx].sum() - tp  # False Positive
    tn = cm.sum() - (tp + fn + fp)  # True Negative

    specificity = tn / (tn + fp) if (tn + fp) > 0 else 0
    sensitivity = tp / (tp + fn) if (tp + fn) > 0 else 0
    precision = report[str(label)]['precision']
    recall = report[str(label)]['recall']
    f1_score = report[str(label)]['f1-score']
    accuracy = report['accuracy']

    scores[label]['specificity'] = specificity
    scores[label]['sensitivity'] = sensitivity
    scores[label]['precision'] = precision
    scores[label]['recall'] = recall
    scores[label]['f1_score'] = f1_score
    scores[label]['accuracy'] = accuracy

# Print evaluation metrics for each label
for label in labels:
    print(f"\nLabel '{label}' Scores:")
    print("Specificity:", scores[label]['specificity'])
    print("Sensitivity:", scores[label]['sensitivity'])
    print("Precision:", scores[label]['precision'])
    print("Recall:", scores[label]['recall'])
    print("F1 Score:", scores[label]['f1_score'])
    print("Accuracy:", scores[label]['accuracy'])

# Summarize results
print("\nOverall accuracy:", report['accuracy'])



Label '0' Scores:
Specificity: 0.973568281938326
Sensitivity: 0.998876404494382
Precision: 0.9932960893854749
Recall: 0.998876404494382
F1 Score: 0.996078431372549
Accuracy: 0.9937332139659804

Label '1' Scores:
Specificity: 0.998876404494382
Sensitivity: 0.973568281938326
Precision: 0.9954954954954955
Recall: 0.973568281938326
F1 Score: 0.9844097995545658
Accuracy: 0.9937332139659804

Overall accuracy: 0.9937332139659804


In [ ]:
# Initialize Neural Network classifier
nn = MLPClassifier(random_state=42, max_iter=1000)

# Initialize KFold cross-validation
kf = KFold(n_splits=5, shuffle=True, random_state=42)

# Get unique labels from the 'Outcome' column
labels = y.unique()

# Dictionary to store evaluation metrics
avg_scores = {label: {'specificity': [], 'sensitivity': [], 'precision': [], 'recall': [], 'f1_score': [], 'accuracy': []} for label in labels}

# Perform KFold cross-validation
for i, (train_index, test_index) in enumerate(kf.split(X)):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    # Fit the classifier on the training data
    nn.fit(X_train, y_train)

    # Predict on the test data
    y_pred = nn.predict(X_test)

    # Generate classification report
    report = classification_report(y_test, y_pred, output_dict=True)

    # Generate confusion matrix
    cm = confusion_matrix(y_test, y_pred, labels=labels)

    # Extract evaluation metrics for each label
    for idx, label in enumerate(labels):
        tp = cm[idx, idx]  # True Positive
        fn = cm[idx, :].sum() - tp  # False Negative
        fp = cm[:, idx].sum() - tp  # False Positive
        tn = cm.sum() - (tp + fn + fp)  # True Negative

        specificity = tn / (tn + fp) if (tn + fp) > 0 else 0
        sensitivity = tp / (tp + fn) if (tp + fn) > 0 else 0
        precision = report[str(label)]['precision']
        recall = report[str(label)]['recall']
        f1_score = report[str(label)]['f1-score']
        accuracy = report['accuracy']

        avg_scores[label]['specificity'].append(specificity)
        avg_scores[label]['sensitivity'].append(sensitivity)
        avg_scores[label]['precision'].append(precision)
        avg_scores[label]['recall'].append(recall)
        avg_scores[label]['f1_score'].append(f1_score)
        avg_scores[label]['accuracy'].append(accuracy)

    # Print evaluation metrics for each label after each iteration
    print(f"Fold {i+1}:")
    for label in labels:
        print(f"\nLabel '{label}' Scores:")
        print("Specificity:", avg_scores[label]['specificity'][-1])
        print("Sensitivity:", avg_scores[label]['sensitivity'][-1])
        print("Precision:", avg_scores[label]['precision'][-1])
        print("Recall:", avg_scores[label]['recall'][-1])
        print("F1 Score:", avg_scores[label]['f1_score'][-1])
        print("Accuracy:", avg_scores[label]['accuracy'][-1])

# Summarize results across all folds
print("\nAverage scores across all folds:")
for label in labels:
    print(f"\nLabel '{label}' Average Scores:")
    print("Specificity:", sum(avg_scores[label]['specificity']) / len(avg_scores[label]['specificity']))
    print("Sensitivity:", sum(avg_scores[label]['sensitivity']) / len(avg_scores[label]['sensitivity']))
    print("Precision:", sum(avg_scores[label]['precision']) / len(avg_scores[label]['precision']))
    print("Recall:", sum(avg_scores[label]['recall']) / len(avg_scores[label]['recall']))
    print("F1 Score:", sum(avg_scores[label]['f1_score']) / len(avg_scores[label]['f1_score']))
    print("Accuracy:", sum(avg_scores[label]['accuracy']) / len(avg_scores[label]['accuracy']))

Fold 1:

Label '0' Scores:
Specificity: 0.9873949579831933
Sensitivity: 0.9943181818181818
Precision: 0.9965831435079726
Recall: 0.9943181818181818
F1 Score: 0.9954493742889646
Accuracy: 0.9928443649373881

Label '1' Scores:
Specificity: 0.9943181818181818
Sensitivity: 0.9873949579831933
Precision: 0.9791666666666666
Recall: 0.9873949579831933
F1 Score: 0.9832635983263599
Accuracy: 0.9928443649373881
Fold 2:

Label '0' Scores:
Specificity: 1.0
Sensitivity: 0.9954285714285714
Precision: 1.0
Recall: 0.9954285714285714
F1 Score: 0.9977090492554409
Accuracy: 0.9964189794091316

Label '1' Scores:
Specificity: 0.9954285714285714
Sensitivity: 1.0
Precision: 0.983739837398374
Recall: 1.0
F1 Score: 0.9918032786885246
Accuracy: 0.9964189794091316
Fold 3:

Label '0' Scores:
Specificity: 0.8949579831932774
Sensitivity: 0.9988623435722411
Precision: 0.9723145071982281
Recall: 0.9988623435722411
F1 Score: 0.9854096520763187
Accuracy: 0.9767233661593554

Label '1' Scores:
Specificity: 0.9988623435722

#Naive Bayes

In [ ]:
from sklearn.naive_bayes import GaussianNB

In [ ]:
# Initialize Naive Bayes classifier (GaussianNB for continuous features)
nb = GaussianNB()

# Fit the classifier on the entire training data
nb.fit(X_train, y_train)

# Predict on the test data
y_pred = nb.predict(X_test)

# Generate classification report
report = classification_report(y_test, y_pred, output_dict=True)

# Generate confusion matrix
cm = confusion_matrix(y_test, y_pred, labels=nb.classes_)

# Dictionary to store evaluation metrics
avg_scores = {label: {'specificity': 0, 'sensitivity': 0, 'precision': 0, 'recall': 0, 'f1_score': 0, 'accuracy': 0} for label in labels}

# Extract evaluation metrics for each label
for idx, label in enumerate(nb.classes_):
    print(f"\nLabel '{label}' Scores:")
    tp = cm[idx, idx]  # True Positive
    fn = cm[idx, :].sum() - tp  # False Negative
    fp = cm[:, idx].sum() - tp  # False Positive
    tn = cm.sum() - (tp + fn + fp)  # True Negative

    specificity = tn / (tn + fp) if (tn + fp) > 0 else 0
    sensitivity = tp / (tp + fn) if (tp + fn) > 0 else 0
    precision = report[str(label)]['precision']
    recall = report[str(label)]['recall']
    f1_score = report[str(label)]['f1-score']
    accuracy = report['accuracy']

    print("Specificity:", specificity)
    print("Sensitivity:", sensitivity)
    print("Precision:", precision)
    print("Recall:", recall)
    print("F1 Score:", f1_score)
    print("Accuracy:", accuracy)

# Summarize results
print("\nOverall accuracy:", report['accuracy'])


Label '0' Scores:
Specificity: 0.9911894273127754
Sensitivity: 0.8629213483146068
Precision: 0.9974025974025974
Recall: 0.8629213483146068
F1 Score: 0.925301204819277
Accuracy: 0.8889883616830797

Label '1' Scores:
Specificity: 0.8629213483146068
Sensitivity: 0.9911894273127754
Precision: 0.6484149855907781
Recall: 0.9911894273127754
F1 Score: 0.7839721254355402
Accuracy: 0.8889883616830797

Overall accuracy: 0.8889883616830797


In [ ]:
# Initialize Naive Bayes classifier
nb = GaussianNB()

# Initialize KFold cross-validation
kf = KFold(n_splits=5, shuffle=True, random_state=42)

# Get unique labels from the 'Outcome' column
labels = y.unique()

# Dictionary to store evaluation metrics
avg_scores = {label: {'specificity': [], 'sensitivity': [], 'precision': [], 'recall': [], 'f1_score': [], 'accuracy': []} for label in labels}

# Perform KFold cross-validation
for i, (train_index, test_index) in enumerate(kf.split(X)):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    # Fit the classifier on the training data
    nb.fit(X_train, y_train)

    # Predict on the test data
    y_pred = nb.predict(X_test)

    # Generate classification report
    report = classification_report(y_test, y_pred, output_dict=True)

    # Generate confusion matrix
    cm = confusion_matrix(y_test, y_pred, labels=labels)

    # Extract evaluation metrics for each label
    for idx, label in enumerate(labels):
        tp = cm[idx, idx]  # True Positive
        fn = cm[idx, :].sum() - tp  # False Negative
        fp = cm[:, idx].sum() - tp  # False Positive
        tn = cm.sum() - (tp + fn + fp)  # True Negative

        specificity = tn / (tn + fp) if (tn + fp) > 0 else 0
        sensitivity = tp / (tp + fn) if (tp + fn) > 0 else 0
        precision = report[str(label)]['precision']
        recall = report[str(label)]['recall']
        f1_score = report[str(label)]['f1-score']
        accuracy = report['accuracy']

        avg_scores[label]['specificity'].append(specificity)
        avg_scores[label]['sensitivity'].append(sensitivity)
        avg_scores[label]['precision'].append(precision)
        avg_scores[label]['recall'].append(recall)
        avg_scores[label]['f1_score'].append(f1_score)
        avg_scores[label]['accuracy'].append(accuracy)

    # Print evaluation metrics for each label after each iteration
    print(f"Fold {i+1}:")
    for label in labels:
        print(f"\nLabel '{label}' Scores:")
        print("Specificity:", avg_scores[label]['specificity'][-1])
        print("Sensitivity:", avg_scores[label]['sensitivity'][-1])
        print("Precision:", avg_scores[label]['precision'][-1])
        print("Recall:", avg_scores[label]['recall'][-1])
        print("F1 Score:", avg_scores[label]['f1_score'][-1])
        print("Accuracy:", avg_scores[label]['accuracy'][-1])

# Summarize results across all folds
print("\nAverage scores across all folds:")
for label in labels:
    print(f"\nLabel '{label}' Average Scores:")
    print("Specificity:", sum(avg_scores[label]['specificity']) / len(avg_scores[label]['specificity']))
    print("Sensitivity:", sum(avg_scores[label]['sensitivity']) / len(avg_scores[label]['sensitivity']))
    print("Precision:", sum(avg_scores[label]['precision']) / len(avg_scores[label]['precision']))
    print("Recall:", sum(avg_scores[label]['recall']) / len(avg_scores[label]['recall']))
    print("F1 Score:", sum(avg_scores[label]['f1_score']) / len(avg_scores[label]['f1_score']))
    print("Accuracy:", sum(avg_scores[label]['accuracy']) / len(avg_scores[label]['accuracy']))


Fold 1:

Label '0' Scores:
Specificity: 1.0
Sensitivity: 0.8579545454545454
Precision: 1.0
Recall: 0.8579545454545454
F1 Score: 0.9235474006116208
Accuracy: 0.8881932021466905

Label '1' Scores:
Specificity: 0.8579545454545454
Sensitivity: 1.0
Precision: 0.6556473829201102
Recall: 1.0
F1 Score: 0.7920133111480865
Accuracy: 0.8881932021466905
Fold 2:

Label '0' Scores:
Specificity: 1.0
Sensitivity: 0.8594285714285714
Precision: 1.0
Recall: 0.8594285714285714
F1 Score: 0.9244007375537799
Accuracy: 0.8898836168307968

Label '1' Scores:
Specificity: 0.8594285714285714
Sensitivity: 1.0
Precision: 0.663013698630137
Recall: 1.0
F1 Score: 0.7973640856672158
Accuracy: 0.8898836168307968
Fold 3:

Label '0' Scores:
Specificity: 1.0
Sensitivity: 0.856655290102389
Precision: 1.0
Recall: 0.856655290102389
F1 Score: 0.9227941176470588
Accuracy: 0.8871978513876455

Label '1' Scores:
Specificity: 0.856655290102389
Sensitivity: 1.0
Precision: 0.6538461538461539
Recall: 1.0
F1 Score: 0.7906976744186047
A

#SVM Linear

In [ ]:
from sklearn.svm import SVC

In [ ]:
# Initialize SVM linear classifier
svm = SVC(kernel='linear')


# Fit the classifier on the training data
svm.fit(X_train, y_train)

# Predict on the test data
y_pred = svm.predict(X_test)

# Generate classification report
report = classification_report(y_test, y_pred, output_dict=True)

# Generate confusion matrix
labels = y.unique()
cm = confusion_matrix(y_test, y_pred, labels=labels)

# Print evaluation metrics
# Iterate over indices and labels simultaneously
for idx, label in enumerate(labels):
    print(f"\nLabel '{label}' Scores:")
    tp = cm[idx, idx]  # True Positive
    fn = cm[idx, :].sum() - tp  # False Negative
    fp = cm[:, idx].sum() - tp  # False Positive
    tn = cm.sum() - (tp + fn + fp)  # True Negative

    specificity = tn / (tn + fp) if (tn + fp) > 0 else 0
    sensitivity = tp / (tp + fn) if (tp + fn) > 0 else 0
    precision = report[str(label)]['precision']
    recall = report[str(label)]['recall']
    f1_score = report[str(label)]['f1-score']
    accuracy = report['accuracy']

    print("Specificity:", specificity)
    print("Sensitivity:", sensitivity)
    print("Precision:", precision)
    print("Recall:", recall)
    print("F1 Score:", f1_score)
    print("Accuracy:", accuracy)
# Summarize results
print("\nOverall accuracy:", report['accuracy'])


Label '0' Scores:
Specificity: 1.0
Sensitivity: 0.9977528089887641
Precision: 1.0
Recall: 0.9977528089887641
F1 Score: 0.998875140607424
Accuracy: 0.9982094897045658

Label '1' Scores:
Specificity: 0.9977528089887641
Sensitivity: 1.0
Precision: 0.9912663755458515
Recall: 1.0
F1 Score: 0.9956140350877193
Accuracy: 0.9982094897045658

Overall accuracy: 0.9982094897045658


In [ ]:
# Initialize SVM classifier with a linear kernel
svm = SVC(kernel='linear')

# Initialize KFold cross-validation
kf = KFold(n_splits=5, shuffle=True, random_state=42)

# Get unique labels from the 'Outcome' column
labels = y.unique()

# Dictionary to store evaluation metrics
avg_scores = {label: {'specificity': [], 'sensitivity': [], 'precision': [], 'recall': [], 'f1_score': [], 'accuracy': []} for label in labels}

# Perform KFold cross-validation
for i, (train_index, test_index) in enumerate(kf.split(X)):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    # Fit the classifier on the training data
    svm.fit(X_train, y_train)

    # Predict on the test data
    y_pred = svm.predict(X_test)

    # Generate classification report
    report = classification_report(y_test, y_pred, output_dict=True)

    # Generate confusion matrix
    cm = confusion_matrix(y_test, y_pred, labels=labels)

    # Extract evaluation metrics for each label
    for idx, label in enumerate(labels):
        tp = cm[idx, idx]  # True Positive
        fn = cm[idx, :].sum() - tp  # False Negative
        fp = cm[:, idx].sum() - tp  # False Positive
        tn = cm.sum() - (tp + fn + fp)  # True Negative

        specificity = tn / (tn + fp) if (tn + fp) > 0 else 0
        sensitivity = tp / (tp + fn) if (tp + fn) > 0 else 0
        precision = report[str(label)]['precision']
        recall = report[str(label)]['recall']
        f1_score = report[str(label)]['f1-score']
        accuracy = report['accuracy']

        avg_scores[label]['specificity'].append(specificity)
        avg_scores[label]['sensitivity'].append(sensitivity)
        avg_scores[label]['precision'].append(precision)
        avg_scores[label]['recall'].append(recall)
        avg_scores[label]['f1_score'].append(f1_score)
        avg_scores[label]['accuracy'].append(accuracy)

    # Print evaluation metrics for each label after each iteration
    print(f"Fold {i+1}:")
    for label in labels:
        print(f"\nLabel '{label}' Scores:")
        print("Specificity:", avg_scores[label]['specificity'][-1])
        print("Sensitivity:", avg_scores[label]['sensitivity'][-1])
        print("Precision:", avg_scores[label]['precision'][-1])
        print("Recall:", avg_scores[label]['recall'][-1])
        print("F1 Score:", avg_scores[label]['f1_score'][-1])
        print("Accuracy:", avg_scores[label]['accuracy'][-1])

# Summarize results across all folds
print("\nAverage scores across all folds:")
for label in labels:
    print(f"\nLabel '{label}' Average Scores:")
    print("Specificity:", sum(avg_scores[label]['specificity']) / len(avg_scores[label]['specificity']))
    print("Sensitivity:", sum(avg_scores[label]['sensitivity']) / len(avg_scores[label]['sensitivity']))
    print("Precision:", sum(avg_scores[label]['precision']) / len(avg_scores[label]['precision']))
    print("Recall:", sum(avg_scores[label]['recall']) / len(avg_scores[label]['recall']))
    print("F1 Score:", sum(avg_scores[label]['f1_score']) / len(avg_scores[label]['f1_score']))
    print("Accuracy:", sum(avg_scores[label]['accuracy']) / len(avg_scores[label]['accuracy']))


Fold 1:

Label '0' Scores:
Specificity: 1.0
Sensitivity: 1.0
Precision: 1.0
Recall: 1.0
F1 Score: 1.0
Accuracy: 1.0

Label '1' Scores:
Specificity: 1.0
Sensitivity: 1.0
Precision: 1.0
Recall: 1.0
F1 Score: 1.0
Accuracy: 1.0
Fold 2:

Label '0' Scores:
Specificity: 1.0
Sensitivity: 1.0
Precision: 1.0
Recall: 1.0
F1 Score: 1.0
Accuracy: 1.0

Label '1' Scores:
Specificity: 1.0
Sensitivity: 1.0
Precision: 1.0
Recall: 1.0
F1 Score: 1.0
Accuracy: 1.0
Fold 3:

Label '0' Scores:
Specificity: 1.0
Sensitivity: 1.0
Precision: 1.0
Recall: 1.0
F1 Score: 1.0
Accuracy: 1.0

Label '1' Scores:
Specificity: 1.0
Sensitivity: 1.0
Precision: 1.0
Recall: 1.0
F1 Score: 1.0
Accuracy: 1.0
Fold 4:

Label '0' Scores:
Specificity: 1.0
Sensitivity: 1.0
Precision: 1.0
Recall: 1.0
F1 Score: 1.0
Accuracy: 1.0

Label '1' Scores:
Specificity: 1.0
Sensitivity: 1.0
Precision: 1.0
Recall: 1.0
F1 Score: 1.0
Accuracy: 1.0
Fold 5:

Label '0' Scores:
Specificity: 1.0
Sensitivity: 0.9977528089887641
Precision: 1.0
Recall: 0.997

#SVM Non-linear

In [ ]:
# Initialize SVM classifier with a non-linear kernel (e.g., 'rbf')
svm = SVC(kernel='rbf')

# Fit the classifier on the entire dataset
svm.fit(X, y)

# Predict on the same dataset
y_pred = svm.predict(X)

# Generate classification report
report = classification_report(y, y_pred, output_dict=True)

# Get unique labels from the 'Outcome' column
labels = y.unique()

# Dictionary to store evaluation metrics
avg_scores = {label: {'specificity': 0, 'sensitivity': 0, 'precision': 0, 'recall': 0, 'f1_score': 0, 'accuracy': 0} for label in labels}

# Generate confusion matrix
cm = confusion_matrix(y, y_pred, labels=labels)

# Extract evaluation metrics for each label
for idx, label in enumerate(labels):
    tp = cm[idx, idx]  # True Positive
    fn = cm[idx, :].sum() - tp  # False Negative
    fp = cm[:, idx].sum() - tp  # False Positive
    tn = cm.sum() - (tp + fn + fp)  # True Negative

    specificity = tn / (tn + fp) if (tn + fp) > 0 else 0
    sensitivity = tp / (tp + fn) if (tp + fn) > 0 else 0
    precision = report[str(label)]['precision']
    recall = report[str(label)]['recall']
    f1_score = report[str(label)]['f1-score']
    accuracy = report['accuracy']


# Print evaluation metrics for each label
print("Evaluation Metrics for SVM Non-Linear Classification:")
for label in labels:
    print(f"\nLabel '{label}' Scores:")
    print("Specificity:", specificity)
    print("Sensitivity:", sensitivity)
    print("Precision:", precision)
    print("Recall:", recall)
    print("F1 Score:", f1_score)
    print("Accuracy:", accuracy)

# Summarize results
print("\nOverall accuracy:", report['accuracy'])


Evaluation Metrics for SVM Non-Linear Classification:

Label '0' Scores:
Specificity: 0.9997722614438624
Sensitivity: 0.9933054393305439
Precision: 0.9991582491582491
Recall: 0.9933054393305439
F1 Score: 0.9962232480067142
Accuracy: 0.9983888292158969

Label '1' Scores:
Specificity: 0.9997722614438624
Sensitivity: 0.9933054393305439
Precision: 0.9991582491582491
Recall: 0.9933054393305439
F1 Score: 0.9962232480067142
Accuracy: 0.9983888292158969

Overall accuracy: 0.9983888292158969


In [ ]:
# Initialize SVM classifier with a non-linear kernel (RBF)
svm = SVC(kernel='rbf', probability=True)

# Initialize KFold cross-validation
kf = KFold(n_splits=5, shuffle=True, random_state=42)

# Get unique labels from the 'Outcome' column
labels = y.unique()

# Dictionary to store evaluation metrics
avg_scores = {label: {'specificity': [], 'sensitivity': [], 'precision': [], 'recall': [], 'f1_score': [], 'accuracy': []} for label in labels}

# Perform KFold cross-validation
for i, (train_index, test_index) in enumerate(kf.split(X)):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    # Fit the classifier on the training data
    svm.fit(X_train, y_train)

    # Predict on the test data
    y_pred = svm.predict(X_test)

    # Generate classification report
    report = classification_report(y_test, y_pred, output_dict=True)

    # Generate confusion matrix
    cm = confusion_matrix(y_test, y_pred, labels=labels)

    # Extract evaluation metrics for each label
    for idx, label in enumerate(labels):
        tp = cm[idx, idx]  # True Positive
        fn = cm[idx, :].sum() - tp  # False Negative
        fp = cm[:, idx].sum() - tp  # False Positive
        tn = cm.sum() - (tp + fn + fp)  # True Negative

        specificity = tn / (tn + fp) if (tn + fp) > 0 else 0
        sensitivity = tp / (tp + fn) if (tp + fn) > 0 else 0
        precision = report[str(label)]['precision']
        recall = report[str(label)]['recall']
        f1_score = report[str(label)]['f1-score']
        accuracy = report['accuracy']

        avg_scores[label]['specificity'].append(specificity)
        avg_scores[label]['sensitivity'].append(sensitivity)
        avg_scores[label]['precision'].append(precision)
        avg_scores[label]['recall'].append(recall)
        avg_scores[label]['f1_score'].append(f1_score)
        avg_scores[label]['accuracy'].append(accuracy)

    # Print evaluation metrics for each label after each iteration
    print(f"Fold {i+1}:")
    for label in labels:
        print(f"\nLabel '{label}' Scores:")
        print("Specificity:", avg_scores[label]['specificity'][-1])
        print("Sensitivity:", avg_scores[label]['sensitivity'][-1])
        print("Precision:", avg_scores[label]['precision'][-1])
        print("Recall:", avg_scores[label]['recall'][-1])
        print("F1 Score:", avg_scores[label]['f1_score'][-1])
        print("Accuracy:", avg_scores[label]['accuracy'][-1])

# Summarize results across all folds
print("\nAverage scores across all folds:")
for label in labels:
    print(f"\nLabel '{label}' Average Scores:")
    print("Specificity:", sum(avg_scores[label]['specificity']) / len(avg_scores[label]['specificity']))
    print("Sensitivity:", sum(avg_scores[label]['sensitivity']) / len(avg_scores[label]['sensitivity']))
    print("Precision:", sum(avg_scores[label]['precision']) / len(avg_scores[label]['precision']))
    print("Recall:", sum(avg_scores[label]['recall']) / len(avg_scores[label]['recall']))
    print("F1 Score:", sum(avg_scores[label]['f1_score']) / len(avg_scores[label]['f1_score']))
    print("Accuracy:", sum(avg_scores[label]['accuracy']) / len(avg_scores[label]['accuracy']))


Fold 1:

Label '0' Scores:
Specificity: 0.9915966386554622
Sensitivity: 1.0
Precision: 0.9977324263038548
Recall: 1.0
F1 Score: 0.9988649262202043
Accuracy: 0.998211091234347

Label '1' Scores:
Specificity: 1.0
Sensitivity: 0.9915966386554622
Precision: 1.0
Recall: 0.9915966386554622
F1 Score: 0.9957805907172996
Accuracy: 0.998211091234347
Fold 2:

Label '0' Scores:
Specificity: 1.0
Sensitivity: 1.0
Precision: 1.0
Recall: 1.0
F1 Score: 1.0
Accuracy: 1.0

Label '1' Scores:
Specificity: 1.0
Sensitivity: 1.0
Precision: 1.0
Recall: 1.0
F1 Score: 1.0
Accuracy: 1.0
Fold 3:

Label '0' Scores:
Specificity: 0.9957983193277311
Sensitivity: 1.0
Precision: 0.9988636363636364
Recall: 1.0
F1 Score: 0.9994314951677089
Accuracy: 0.999104744852283

Label '1' Scores:
Specificity: 1.0
Sensitivity: 0.9957983193277311
Precision: 1.0
Recall: 0.9957983193277311
F1 Score: 0.9978947368421053
Accuracy: 0.999104744852283
Fold 4:

Label '0' Scores:
Specificity: 0.984
Sensitivity: 1.0
Precision: 0.9954075774971297